# Secondo set di  Esercizi

#### Federico Schipani, 6185896, [federico.schipani@stud.unifi.it](mailto:federico.schipani@stud.unifi.it)

##### Prerequisiti
Per eseguire in maniera interattiva questo notebook è necessario avviare un server Jupyter. Se si fa uso dell'interprete Anaconda questo risulta già installato, sennò è necessario installarlo con il comando:

```
pip install jupyter
```
Una volta completata l'installazione per avviare il server è sufficiente aprire un terminale, posizionarsi nella cartella dove è stato salvato questo notebook e dare il comando
```
jupyter notebook
```
successivamente si aprirà una finestra del browser dove è possibile selezionare il notebook da eseguire.

Per questo notebook è stato fatto uso di una [libreria](https://github.com/jupyter-widgets/ipywidgets) per generare widget interattivi in HTML, per installare questa libreria è sufficiente dare i seguenti due comandi sul terminale:
```
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
```
Se dovessero esserci problemi con la visualizzazione dei widgets:
```
jupyter nbextension install --py widgetsnbextension
jupyter nbextension enable jupyter-js-widgets/extension
```

### Primo esercizio di programmazione

Il seguente esercizio richiedeva innanzitutto di implementare cinque algoritmi:
1. Algoritmo di Euclide esteso
2. Algoritmo di esponenziazione modulare veloce
3. Test di Miller-Rabin
4. Algoritmo per la generazione di numeri primi
5. Schema RSA, con e senza ottimizzazione CRT


Vediamo le varie implementazioni in ordine. Partiamo innanzitutto importando le librerie necessarie.

In [1]:
import math
import random
import timeit
from ipywidgets import widgets, interact_manual, interact, interactive, fixed
from IPython.display import display
import sys
from IPython.core.display import clear_output
def change_output(x):
    clear_output()
    sys.stdout.write(str(x))
    sys.stdout.flush()

#### Algoritmo di Euclide esteso
Il codice che mostra l'algoritmo di Euclide Esteso è mostrato nel box sottostante. Questo algoritmo calcola l'MCD tra il numero $a$ e il numero $b$ e restituisce una tupla dove il primo elemento è l'MCD, ed il secondo è l'eventuale inverso di $a\ mod\ b$.
Questo inverso esiste solo se l'MCD è uguale a $1$.

In [2]:
def extended_euclidean_algorithm(a, b):
    rm = b
    rm1 = a
    qm1 = 1
    t = 0
    while rm1 != 0:
        q = math.floor(rm / rm1)
        temp = t
        t = qm1
        qm1 = temp - q * qm1
        temp = rm
        rm = rm1
        rm1 = temp - q * rm1
    if t < 0:
        t = t + b
    if rm>1:
        t = "ND"
    return rm, t
euclidean_w = interact(extended_euclidean_algorithm, a = widgets.IntText(description = "a: "), b = widgets.IntText(description = "b: "))

(0, 0)

#### Algoritmo di esponenziazione modulare veloce

Questo algoritmo permette di calcolare in maniera efficiente e veloce il risultato di $a^{n}\ mod\ m$.

In [24]:
def fast_exp_alg(a, n, m):
    d, c = 1, 0
    bin_n = "{0:b}".format(n)
    for i in bin_n:
        d = (d * d) % m
        if int(i) == 1:
            d = (d * a) % m
    return d
aw = widgets.IntText(description  = "a: ", value = 3)
nw = widgets.IntText(description = "n: ", value = 3)
mw = widgets.IntText(description = "m: ", value = 3)
to_display = interactive(fast_exp_alg, a = aw, n = nw, m = mw)
display(to_display)

2

#### Test di Miller-Rabin

Il test di Miller-Rabin serve per verificare se un numero è composto. Questo test richiede due argomenti $x$ ed $n$. Se il valore di ritorno è $true$ allora siamo certi che il numero $x$ sia composto, e diciamo che $n$ è testimone di Rabin per $x$.
Se invece il test ritorna il valore $false$ non possiamo dire con certezza che il numero non sia composto, quindi occorre rieseguire un numero di volte abbastanza elevato per ridurre la probabilità di errore. La probabilità che il test restituisca $false$ con $x$ composto è di $\frac{1}{4}$, quindi se si esegue il test $m$ volte con $n$ diversi la probabilità si riduce a $\frac{1}{4^{m}} = 4^{-m}$.

In [4]:
def rabin_test(x, n):
    m, r, xr = n - 1, 0, []
    while m % 2 == 0:
        m = m // 2
        r = r + 1
    xr.append(fast_exp_alg(x, m, n))
    for i in range(1, r + 1):
        xr.append(fast_exp_alg(xr[i - 1], 2, n))
    return (xr[0] != 1) and all(xi % n != n - 1 for xi in xr[0:-1])
rabin_test_w = interact(rabin_test, x = widgets.IntText(description = "x: ", value = 3), n = widgets.IntText(description = "n: ", value = 2))

False

#### Algoritmo per la generazione di numeri primi

Questo algoritmo fa uso del test di Miller-Rabin per la generazione di un numero  primo di dimensione massima $10^{limit}$.
L'algoritmo richiede un altro paramentro, ovvero $accuracy$, questo parametro non è altro che un intero che serve ad indicare quante volte ripetere il test di Miller-Rabin per il numero generato casualmente. 

In [5]:
def generate_random_prime(minimum, limit, accuracy):
    random_number = 0
    condition = True
    while condition:
        random_number = random.randint(minimum, limit)
        if random_number % 2 != 0:
            test_sample = [random.randint(2, limit) for i in range(0, accuracy)]
            condition = any(rabin_test(x, random_number) for x in test_sample)
    return random_number

prime_slider_limit_1 = widgets.IntSlider(min = 2, max = 100, step = 1, description = "numero di cifre: ")
prime_slider_accuracy_1 = widgets.IntSlider(min = 1, max = 50, step = 1, description = "accuracy: ")
prime_button_generate = widgets.Button(description = "Genera")
def run_test_prime_1(b):
    print(generate_random_prime(10**(prime_slider_limit_1.value-1),(10**(prime_slider_limit_1.value)-1) , prime_slider_accuracy_1.value))
prime_button_generate.on_click(run_test_prime_1)
display(prime_slider_limit_1)
display(prime_slider_accuracy_1)
display(prime_button_generate)

Verifichiamo ora i tempi di esecuzione dell'algoritmo variando $limit$ e $accuracy$.

In [6]:
prime_slider_limit = widgets.IntSlider(min = 2, max = 100, step = 1, description = "numero di cifre: ")
prime_slider_accuracy = widgets.IntSlider(min = 1, max = 50, step = 1, description = "accuracy: ")
prime_button_run = widgets.Button(description = "Esegui test")
def run_test_prime(b):
    %timeit -n 100 generate_random_prime(10**(prime_slider_limit.value-1), (10**(prime_slider_limit.value)-1), prime_slider_accuracy.value)
prime_button_run.on_click(run_test_prime)
display(prime_slider_limit)
display(prime_slider_accuracy)
display(prime_button_run)

#### RSA

Dopo aver implementato tutti gli algoritmi precedenti siamo pronti per implementare RSA. Sono state implementate due versioni, la prima standard, e la seconda che fa uso delle ottimizzazioni derivate dal Teorema Cinese Del Resto.

Per prima cosa è stata definita una funzione che, dati due numeri primi $p$ e $q$ genera la chiave pubblica e privata. Il primo elemento della tupla che restituisce è la chiave pubblica che verrà usata per criptare il Plain Text, il secondo elemento invece è la chiave privata, che verrà usata per decriptare il Cypher Text.

Durante la descrizione dell'algoritmo si seguirà l'esempio delle Note, per far sì che i risultati siano coerenti con quelli dell'esempio non possiamo generare casualmente il numero primo $d$, quindi verrà ECCEZIONALMENTE passato come input del generatore di chiave.

In [7]:
def generate_rsa_key(p, q, d):
    n = p * q
    phi = (p - 1) * (q - 1)
    if d == 0:
        d = generate_random_prime(2, n-1, 5)
        while extended_euclidean_algorithm(d, phi)[0] != 1:
            d = generate_random_prime(2, n-1, 16)
    e = extended_euclidean_algorithm(d, phi)
    kp = (e[1], n)
    km = (d, n)
    return kp, km
standard_keys = interactive(generate_rsa_key, 
         p = widgets.IntText(description = "p: ", value = 3), 
         q = widgets.IntText(description = "q: ", value = 11), 
         d = widgets.IntText(description = "d: ", value = 7))
display(standard_keys)

((3, 33), (7, 33))

Con le chiavi appena generate è possibile cifrare un Plain Text $m$.


In [8]:
def rsa_encrypt(m, kp):
    return fast_exp_alg(m, kp[0], kp[1])
standard_cypher_text = interactive(rsa_encrypt, m = widgets.IntText(description = "m: ", value = 8), kp = fixed(standard_keys.result[0]))
display(standard_cypher_text)

17

Dato il Cypher Text appena generato proviamo a decriptarlo per vedere se si ottiene lo stesso $m$ passatogli in input.

In [9]:
def rsa_decrypt(c, km):
    return fast_exp_alg(c, km[0], km[1])


standard_decrypt = interactive(rsa_decrypt, c = fixed(standard_cypher_text.result), km = fixed(standard_keys.result[1]))
decrypt_button = widgets.Button(description = "Decrypt")
def decrypt_button_function(b):
    change_output(rsa_decrypt(standard_cypher_text.result, standard_keys.result[1]))
decrypt_button.on_click(decrypt_button_function)
display(standard_decrypt)
display(decrypt_button)

8

Vediamo ora le ottimizzazioni che si possono ottenere attraverso l'uso del Teorema Cinese del resto.

Innanzitutto cambia il modo di generare la chiave, in quanto la chiave privata non è più una tupla, ma una quintupla.
Come prima, per seguire l'esempio delle note è necessario passare in input alla funzione di generazione della chiave un valore prefissato per $d$.

In [10]:
def generate_rsa_crt_key(p, q, d):
    n = p * q
    phi = (p - 1) * (q - 1)
    if d == 0:
        d = generate_random_prime(2, n - 1, 5)
        while d != 1 and extended_euclidean_algorithm(d, phi)[0] != 1:
            d = generate_random_prime(2, n-1, 5)
    q_inv = extended_euclidean_algorithm(q, p)[1]
    p_inv = extended_euclidean_algorithm(p, q)[1]
    e = extended_euclidean_algorithm(d, phi)
    kp = (e[1], n)
    km = (p, q, d, p_inv*p, q_inv*q)
    return kp, km

crt_keys = interactive(generate_rsa_crt_key, 
         p = widgets.IntText(description = "p: ", value = 3), 
         q = widgets.IntText(description = "q: ", value = 11), 
         d = widgets.IntText(description = "d: ", value = 7))
display(crt_keys)

((3, 33), (3, 11, 7, 12, 22))

Visto che la chiave pubblica che si usa per criptare il messaggio ha la stessa forma della chiave pubblica che si usa per la versione senza ottimizzazioni è possibile riusare la stessa funzione per criptare il messaggio.

In [11]:
crt_cypher_text = interactive(rsa_encrypt, m = widgets.IntText(description = "m: ", value = 8), kp = fixed(crt_keys.result[0]))
display(crt_cypher_text)

17

Infatti come si può notare il risultato è lo stesso.

Vediamo ora se funziona tutto eseguendo la decodifica.

In [12]:
def rsa_decrypt_crt(c, km):
    mp = fast_exp_alg(c, km[2], km[0])
    mq = fast_exp_alg(c, km[2], km[1])
    return ((mp * km[4]) + (mq * km[3]))%(km[0]*km[1])


crt_decrypt = interactive(rsa_decrypt_crt, c = fixed(crt_cypher_text.result), km = fixed(crt_keys.result[1]))
decrypt_crt_button = widgets.Button(description = "Decrypt")
def decrypt_crt_button_function(b):
    change_output(rsa_decrypt_crt(crt_cypher_text.result, crt_keys.result[1]))
decrypt_crt_button.on_click(decrypt_crt_button_function)
display(crt_decrypt)
display(decrypt_crt_button)

8

Come richiesto dal testo dell'esercizio effettuiamo un test delle prestazioni, innanzitutto fissiamo un $p$ ed un $q$ di dimensione realistica, e calcoliamo quindi il modulo RSA $n$.
Ovviamente devo fissare anche il valore $d$ in maniera tale da generare lo stesso set di chiavi per la versione CRT e standard.

In [13]:
dim = 300
def generate_test_case(dimension):
    big_p_test = generate_random_prime(10**dimension, (10**(dimension+1))-1, 16)
    big_q_test = generate_random_prime(10**dimension, (10**(dimension+1))-1, 16)
    big_phi_test = (big_p_test - 1) * (big_q_test - 1)
    big_n_test = big_p_test*big_q_test
    big_d_test = generate_random_prime(2, big_n_test - 1, 16)
    while big_d_test != 1 and extended_euclidean_algorithm(big_d_test, big_phi_test)[0] != 1:
        big_d_test = generate_random_prime(2, big_n_test - 1, 16)
    return big_p_test, big_q_test, big_d_test

p_test, q_test, d_test = generate_test_case(dim)

Generiamo quindi la chiave per RSA standard e RSA con ottimizzazioni CRT.

In [14]:
standard_test_keys = generate_rsa_key(p_test, q_test, d_test)
standard_test_keys

((6610036275749217236689302308015543191070775355372271940702108919412930284700719259734382353435525292563469372126863842970768918575272074121939773432667150663660094624423840183319999074651744270958650099438442706874157393101614947069338986654324768934699990905622408733763279286067980490059361287469532805866123581358907060655985296548580357062678009609409979173554398076375783663063071931841777536152517662962524901682416119714059894745465607562152492500530111531431138537869857947857427812402675225909969097196568497493337516948838174089537753074500982792522229705824349643710777302358017762236017103,
  114159763385234629354689061532263785157682156926248587132313799636704913957462635132250951821715205603241258711865198592099929671484389023164763288844691545957469796018165330995651560391024298327622768064422952541069931709421823931503701737906544915132788982601371491632561693305015217813720901947168692777287796593129609956486969924991275907367543362409781145083286459622257251085857219946254

In [15]:
crt_test_keys =  generate_rsa_crt_key(p_test, q_test, d_test)
crt_test_keys

((6610036275749217236689302308015543191070775355372271940702108919412930284700719259734382353435525292563469372126863842970768918575272074121939773432667150663660094624423840183319999074651744270958650099438442706874157393101614947069338986654324768934699990905622408733763279286067980490059361287469532805866123581358907060655985296548580357062678009609409979173554398076375783663063071931841777536152517662962524901682416119714059894745465607562152492500530111531431138537869857947857427812402675225909969097196568497493337516948838174089537753074500982792522229705824349643710777302358017762236017103,
  114159763385234629354689061532263785157682156926248587132313799636704913957462635132250951821715205603241258711865198592099929671484389023164763288844691545957469796018165330995651560391024298327622768064422952541069931709421823931503701737906544915132788982601371491632561693305015217813720901947168692777287796593129609956486969924991275907367543362409781145083286459622257251085857219946254

Effettuiamo ora un test delle prestazioni scegliendo 100 plain text casuali. Per prima cosa generiamo i 100 plain text, successivamente si cifrano.

In [16]:
plain_text_test = [random.randint(1, 10**dim) for i in range(0, 100)]
cypher_text_test = [rsa_encrypt(m, standard_test_keys[0]) for m in plain_text_test]

Prima di effettuare un test sui tempi di decodifica dei messaggi vediamo se l'intera procedura funziona correttamente.

In [17]:
result_vector = [rsa_decrypt(c, standard_test_keys[1]) for c in cypher_text_test]
result_vector_2 = [rsa_decrypt_crt(c, crt_test_keys[1]) for c in cypher_text_test]
result_vector == plain_text_test == result_vector_2

True

Siamo ora pronti per verificare i tempi di esecuzione:

In [18]:
def decrypting_test(cypher_text, keys):
    for c in cypher_text:
        rsa_decrypt(c, keys)
        
standard_time = %timeit -o decrypting_test(cypher_text_test, standard_test_keys[1])

1 loop, best of 3: 3.57 s per loop


Con gli stessi Cypher Text proviamo ora la versione ottimizzata che fa uso del Teorema Cinese del Resto.

In [19]:
def decrypting_test_crt(cypher_text, keys):
    for c in cypher_text:
        rsa_decrypt_crt(c, keys)
        
crt_time = %timeit -o decrypting_test_crt(cypher_text_test, crt_test_keys[1])

1 loop, best of 3: 2.79 s per loop


Facendo una rapida operazione si può calcolare lo speedup in percentuale.

In [23]:
def calc_speedup(time1, time2):
    return ((time1 / time2)-1)*100
#calc_speedup(standard_time.average, crt_time.average) su Windows questa riga di codice non funziona (???)
calc_speedup(standard_time.best, crt_time.best)

27.660632138013398

Come si può notare si ottiene speedup.